<a href="https://colab.research.google.com/github/amazzoli/ComponentSystemsData/blob/main/Load_and_compute_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cloning the repository in the colab folder structure
%%bash
git clone https://github.com/amazzoli/ComponentSystemsData.git

Cloning into 'ComponentSystemsData'...
Filtering content: 100% (2/2), 100.21 MiB | 56.35 MiB/s, done.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys

sys.path.append('/content/ComponentSystemsData/py_utils/')
#import plot_utils as pu

In [ ]:

def ComponentSystem(object):

  """
  Class storing all the properties of a component system and computing the
  main statistical properties
  """

  def __init__(self, objects, components, sparse_counts):
    """
    Args:
    - objects: pandas table with the list of objects.
    - components: pandas table with the list of components.
    - sparse_counts: pandas table with the sparse counts.
    """
    self.objects = objects
    self.components = components
    self.sparse_counts = sparse_counts


  def check_consistency(self):
    """
    Check if the identifiers and the summation of rows and columns in the objects
    and realization tables are consistent with the sparse count matrix.
    """
    pass


compsys_lables = ['books', ]
def load_compsys(label):

